In [1]:
import json
import pandas as pd
import requests
import time

subscription_key = "FOR_SAFETY_REASONS_I_REMOVED_THE_KEY"
search_term = "Tesla"
search_url = "https://api.bing.microsoft.com/v7.0/news/search"

In [2]:
headers = {"Ocp-Apim-Subscription-Key": subscription_key}
params = {
    "q": search_term,
    "textDecorations": True,
    "textFormat": "HTML",
    "cc": "en-US",
    "count": 100,
    "category": "Business",
    "mkt": "en-US",
    "safeSearch": "Moderate",
    "sortBy": "Relevance",
    "Freshness": "Month"
}

Tesla 관련 뉴스 Bing news search API 이용해 크롤링

In [3]:

all_results = []

# Loop to make 500 different requests
for i in range(500):
    # Set the offset parameter
    params["offset"] = i * 100

    try:
        # Make the request
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response JSON and append to all_results
        search_results = response.json()
        all_results.extend(search_results.get("value", []))

        # Optional: Introduce a delay to avoid hitting rate limits
        time.sleep(1)

    except requests.exceptions.RequestException as e:
        # Handle errors (you can print or log the error message)
        print(f"Error in request {i+1}: {e}")
        break

# Convert the results to JSON string
all_results_json = json.dumps(all_results)

# You can now work with all_results or all_results_json
# For example, you can convert it into a pandas DataFrame
# df = pd.DataFrame(all_results)


REST API로 최근 한달간 가장 연관성 높은 기사 5만개 수집

In [9]:
# Convert the results to a pandas DataFrame
df = pd.DataFrame(all_results)
df.head()

,name,url,image,description,about,provider,datePublished,category,mentions,video
0,Modi in US: Elon Musk says <b>Tesla</b> to com...,https://www.bbc.com/news/world-asia-india-6595...,{'thumbnail': {'contentUrl': 'https://www.bing...,Tesla CEO Elon Musk has said the electric carm...,[{'readLink': 'https://api.bing.microsoft.com/...,"[{'_type': 'Organization', 'name': 'BBC', 'ima...",2023-06-21T04:54:00.0000000Z,World,NaN,NaN
1,Musk Says <b>Tesla</b> Considering India Inves...,https://www.msn.com/en-us/money/companies/musk...,{'thumbnail': {'contentUrl': 'https://www.bing...,Tesla Inc. is likely to make a significant inv...,[{'readLink': 'https://api.bing.microsoft.com/...,"[{'_type': 'Organization', 'name': 'Bloomberg ...",2023-06-21T04:27:46.0000000Z,NaN,"[{'name': 'Narendra Modi'}, {'name': 'Elon Mus...",NaN
2,<b>Tesla</b> secret configuration allows selec...,https://www.msn.com/en-us/news/other/tesla-sec...,{'thumbnail': {'contentUrl': 'https://www.bing...,The researcher has nicknamed the feature &quot...,[{'readLink': 'https://api.bing.microsoft.com/...,"[{'_type': 'Organization', 'name': 'CNBC on MS...",2023-06-20T23:40:11.0000000Z,ScienceAndTechnology,"[{'name': 'Secret'}, {'name': 'Tesla Motors'},...",NaN
3,<b>Tesla</b> Rival Kia Rolls Out Details on Ne...,https://www.msn.com/en-us/lifestyle/shopping/t...,{'thumbnail': {'contentUrl': 'https://www.bing...,Kia has gone into expansion mode to offer a co...,[{'readLink': 'https://api.bing.microsoft.com/...,"[{'_type': 'Organization', 'name': 'TheStreet....",2023-06-21T04:43:00.0000000Z,Business,"[{'name': 'Electric vehicle'}, {'name': 'Kia M...",NaN
4,Jim Cramer says he&#39;s betting on Ford but s...,https://www.msn.com/en-us/money/markets/jim-cr...,{'thumbnail': {'contentUrl': 'https://www.bing...,While Cramer still believes investors should o...,[{'readLink': 'https://api.bing.microsoft.com/...,"[{'_type': 'Organization', 'name': 'CNBC on MS...",2023-06-20T23:25:14.0000000Z,Business,"[{'name': 'Jim Cramer'}, {'name': 'Ford Motor ...",{'name': '&#39;I don&#39;t think Ford and Tesl...


JSON Dataframe 으로 변환

In [5]:
df.head()
print(df.shape[0])

41604


실제로는 41604 건의 뉴스 수집됨

In [6]:
df = df[['name', 'description']]

기사 제목과 본문만 선택

In [7]:
df.head()

,name,description
0,Modi in US: Elon Musk says <b>Tesla</b> to com...,Tesla CEO Elon Musk has said the electric carm...
1,Musk Says <b>Tesla</b> Considering India Inves...,Tesla Inc. is likely to make a significant inv...
2,<b>Tesla</b> secret configuration allows selec...,The researcher has nicknamed the feature &quot...
3,<b>Tesla</b> Rival Kia Rolls Out Details on Ne...,Kia has gone into expansion mode to offer a co...
4,Jim Cramer says he&#39;s betting on Ford but s...,While Cramer still believes investors should o...


In [8]:
df.to_csv('bing_news.csv', index=False)

bing_news.csv 파일로 저장, 불용어 처리는 이후 전처리 과정에서 시행